In [2]:
import numpy as np
import pandas as pd
import os
import sys

from subprocess import call, check_output, STDOUT, check_call

# Using `partis` to generate synthetic datasets

We can use the software `partis` to generate synthetic datasets. (**Add link to partis**)

In [87]:
# configuration
partis_path = '/home/fede/src/partis_old'
output_path = '/home/fede/projects_local/davide/data/partis_RAW_v6'
n_iter = 3

In [90]:
# need to install geiger, ape, TreeSim in R for this to work
for i in [1500, 3000, 5000]:
    for j in [400]:
        call('{1}/bin/partis simulate '
             #'--parameter-dir {1}/test/reference-results/test/parameters/data/ '
             '--simulate-partially-from-scratch '
             '--outfname {3}/clones_{0}.{2}.csv --n-sim-events {0} --n-leaves {2} '
             '--indel-frequency 0.05 --indel-location cdr3 --mean-indel-length 6 '
             '--n-procs 16'.format(i, partis_path, j, output_path).split())

This produces into `output_path` folder a list of RAW sequences.

To simplify the processing of IMGT/HighV-Quest, let's a unique `fasta` file where, in the `ID` string, there is also the identity of the original database name. This will allow us to recover our original databases splitted.

In [91]:
path = output_path + '/'
files = [path + x for x in os.listdir(path) if x.endswith('.csv')]

# 1. create a single pandas dataframe
db_s = []
for x in files:
    df = pd.read_csv(x, index_col=None)
    df['db'] = x.split('/')[-1]
    db_s.append(df)

df = pd.concat(db_s)

In [94]:
# 2. create fasta file up to 500k sequences
for i in range(df.shape[0] / 500000 + 1):
    with open(os.path.join(path, "all_{}.fasta".format(i)), 'w') as f:
        for index, row in (df.iloc[i * 500000:(i+1)*500000].iterrows()):
            f.write(">" + "_".join([row['db']] + [str(a) for a in row.values[:-8]]))
            f.write("\n")
            f.write(row['seq'])
            f.write("\n")

Ok! Now we can use IMGT to convert our `fasta` file(s) into databases which we can use as input to ICING.
To do so, connect to IMGT HighV-Quest software and upload the data.

When finished, an email will notify that results are ready. Now, download them and extract the "txz" files as folders to use them with Change-O `MakeDb` script.

In [97]:
%%bash
# run Changeo to convert IMGT into fasta file
# python MakeDb.py imgt -i <imgt output, zip or folder> -s <original fasta file> --scores
for i in {0..12}
  do 
     python /home/fede/Dropbox/projects/davide/changeo/MakeDb.py imgt -i /home/fede/projects_local/davide/data/partis_RAW_v6/imgt-pass/partis_6_$i -s /home/fede/projects_local/davide/data/partis_RAW_v6/fasta/all_$i.fasta --scores
done

        START> MakeDb
      ALIGNER> IMGT
ALIGN_RESULTS> /home/fede/projects_local/davide/data/partis_RAW_v6/imgt-pass/partis_6_0
     SEQ_FILE> all_0.fasta
     NO_PARSE> False
 SCORE_FIELDS> True

PROGRESS> 09:41:56 [####################] 100% (500,000) 2.4 min

OUTPUT> /home/fede/projects_local/davide/data/partis_RAW_v6/imgt-pass/partis_6_0_db-pass.tab
  PASS> 494892
  FAIL> 5108
   END> MakeDb

        START> MakeDb
      ALIGNER> IMGT
ALIGN_RESULTS> /home/fede/projects_local/davide/data/partis_RAW_v6/imgt-pass/partis_6_1
     SEQ_FILE> all_1.fasta
     NO_PARSE> False
 SCORE_FIELDS> True

PROGRESS> 09:44:37 [####################] 100% (500,000) 2.5 min

OUTPUT> /home/fede/projects_local/davide/data/partis_RAW_v6/imgt-pass/partis_6_1_db-pass.tab
  PASS> 493651
  FAIL> 6349
   END> MakeDb

        START> MakeDb
      ALIGNER> IMGT
ALIGN_RESULTS> /home/fede/projects_local/davide/data/partis_RAW_v6/imgt-pass/partis_6_2
     SEQ_FILE> all_2.fasta
     NO_PARSE> False
 SCORE_FIELDS> Tru

ERROR:  Sequence file /home/fede/projects_local/davide/data/partis_RAW_v6/fasta/all_9.fasta does not exist
ERROR:  Sequence file /home/fede/projects_local/davide/data/partis_RAW_v6/fasta/all_10.fasta does not exist
ERROR:  Sequence file /home/fede/projects_local/davide/data/partis_RAW_v6/fasta/all_11.fasta does not exist
ERROR:  Sequence file /home/fede/projects_local/davide/data/partis_RAW_v6/fasta/all_12.fasta does not exist


Divide now the IMGT-ChangeO processed files into a final list of databases which are usable from our method.

In [98]:
path = '/home/fede/projects_local/davide/data/partis_RAW_v6/makedb-pass/'
db_s = []
for f in [os.path.join(path, x) for x in os.listdir(path) if x.endswith('db-pass.tab')]:
    db_s.append(pd.read_csv(f, dialect='excel-tab'))

df = pd.concat(db_s)
    
# add the mut column
df['MUT'] = (1 - df['V_IDENTITY']) * 100.

df['DB'] = df['SEQUENCE_ID'].str.split('.csv').apply(lambda x: min(x, key=len))
for i in df.DB.unique():
    df[df.DB == i].to_csv(os.path.join(path, str(i) + '.tab'), index=False, sep='\t')

Let's produce an overview of the datasets.

In [67]:
from icing.utils import io
path = '/home/fede/projects_local/davide/data/partis_RAW_v6/datasets/'
from icing.externals.DbCore import parseAllele, gene_regex, junction_re

df_all = pd.DataFrame()
for f in sorted([os.path.join(path, x) for x in os.listdir(path) if x.startswith('clones_')]):
    df = io.load_dataframe(f)
    df['true_clone'] = [x[3] for x in df.sequence_id.str.split('_')] 
    row = {}
    
    df['true_v'] = [parseAllele(x[4], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
    df = df.loc[['OR' not in x for x in df.true_v], :]
        
    df['true_d'] = [parseAllele(x[5], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
    df = df.loc[['OR' not in x for x in df.true_d], :]
    
    df['true_j'] = [parseAllele(x[6], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
#     df = df.iloc[:1053694]

    row['unique V genes'] = int(df.true_v.unique().size)
    row['unique D genes'] = int(df.true_d.unique().size)
    row['unique J genes'] = int(df.true_j.unique().size)
    
    row['unique functional V genes'] = len([x for x in set(df.true_v) if 'OR' not in x])
    row['unique functional D genes'] = len([x for x in set(df.true_d) if 'OR' not in x])
    row['unique functional J genes'] = len([x for x in set(df.true_j) if 'OR' not in x])
    
    row['database'] = f.split('/')[-1]
    row['n_seqs'] = int(df.shape[0])
    row['clonotypes'] = int(df.true_clone.unique().size)
    row['avg seqs/clone'] = np.mean([len(x) for x in df.groupby('true_clone').groups.values()])
        
    row['mean (std) of V gene mutation'] = "%.2f (%.2f)" % (df.mut.mean(), df.mut.std())
    df_all = df_all.append(row, ignore_index=True)

In [71]:
df_all['indexNumber'] = [int(i.split('_')[-1].split('.')[0]) + int(
    i.split('_')[-1].split('.')[1]) for i in df_all.database]
# Perform sort of the rows
df_all.sort_values(['indexNumber'], ascending = [True], inplace = True)
# Deletion of the added column
df_all.drop('indexNumber', 1, inplace = True)

df_all['avg seqs/clone'] = df_all['avg seqs/clone'].map('{:.2f}'.format)

df_all[['n_seqs', 'clonotypes', 'unique V genes', 'unique D genes', 'unique J genes',
       'unique functional V genes','unique functional D genes','unique functional J genes']] = df_all[
    ['n_seqs', 'clonotypes', 'unique V genes', 'unique D genes', 'unique J genes',
     'unique functional V genes','unique functional D genes','unique functional J genes']].astype(int)

sorted_df = df_all.loc[:, ['database', 'n_seqs', 'clonotypes', 'avg seqs/clone', 'unique V genes',
               'unique D genes', 'unique J genes',
                           'mean (std) of V gene mutation']]

In [72]:
sorted_df

,database,n_seqs,clonotypes,avg seqs/clone,unique V genes,unique D genes,unique J genes,mean (std) of V gene mutation
0,clones_100.100.tab,7111,77,92.35,35,24,6,9.59 (4.64)
1,clones_100.200.tab,13697,74,185.09,38,24,6,8.64 (4.46)
2,clones_100.400.tab,30525,77,396.43,34,25,6,9.04 (4.51)
9,clones_500.100.tab,38542,389,99.08,56,25,6,8.63 (4.30)
10,clones_500.200.tab,81264,388,209.44,58,25,6,8.41 (4.70)
11,clones_500.400.tab,162379,379,428.44,56,25,6,9.56 (4.46)
3,clones_1500.100.tab,128082,1168,109.66,58,25,6,8.72 (4.67)
4,clones_1500.200.tab,243337,1180,206.22,58,25,6,9.15 (4.73)
5,clones_1500.400.tab,474307,1185,400.26,58,25,6,8.94 (4.65)
6,clones_3000.100.tab,219729,2282,96.29,58,25,6,8.84 (4.46)


In [73]:
sorted_df.to_latex("/home/fede/Dropbox/projects/icing/cibb17/dataset_table.tex", index=False)

In [48]:
df = io.load_dataframe(sorted([os.path.join(path, x) for x in os.listdir(path) if x.startswith('clones_')])[-1])

IndexError: list index out of range

In [134]:
df['true_v'] = [parseAllele(x[4], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
df['true_d'] = [parseAllele(x[5], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 
df['true_j'] = [parseAllele(x[6], gene_regex, 'first') for x in df.sequence_id.str.split('_')] 

In [119]:
len([x for x in set(df.true_v) if 'OR' not in x])

58

In [135]:
set(df.true_v)

{'IGHV1-18',
 'IGHV1-2',
 'IGHV1-24',
 'IGHV1-3',
 'IGHV1-45',
 'IGHV1-46',
 'IGHV1-58',
 'IGHV1-69',
 'IGHV1-69-2',
 'IGHV1-8',
 'IGHV1/OR15-1',
 'IGHV1/OR15-5',
 'IGHV1/OR15-9',
 'IGHV1/OR21-1',
 'IGHV2-26',
 'IGHV2-5',
 'IGHV2-70',
 'IGHV2-70D',
 'IGHV2/OR16-5',
 'IGHV3-11',
 'IGHV3-13',
 'IGHV3-15',
 'IGHV3-16',
 'IGHV3-20',
 'IGHV3-21',
 'IGHV3-23',
 'IGHV3-23D',
 'IGHV3-25',
 'IGHV3-30',
 'IGHV3-30-3',
 'IGHV3-33',
 'IGHV3-35',
 'IGHV3-38',
 'IGHV3-38-3',
 'IGHV3-43',
 'IGHV3-43D',
 'IGHV3-48',
 'IGHV3-49',
 'IGHV3-53',
 'IGHV3-64',
 'IGHV3-64D',
 'IGHV3-66',
 'IGHV3-7',
 'IGHV3-72',
 'IGHV3-73',
 'IGHV3-74',
 'IGHV3-9',
 'IGHV3-NL1',
 'IGHV3/OR15-7',
 'IGHV3/OR16-10',
 'IGHV3/OR16-12',
 'IGHV3/OR16-13',
 'IGHV3/OR16-6',
 'IGHV3/OR16-8',
 'IGHV3/OR16-9',
 'IGHV4-28',
 'IGHV4-30-2',
 'IGHV4-30-4',
 'IGHV4-31',
 'IGHV4-34',
 'IGHV4-38-2',
 'IGHV4-39',
 'IGHV4-4',
 'IGHV4-59',
 'IGHV4-61',
 'IGHV4/OR15-8',
 'IGHV5-10-1',
 'IGHV5-51',
 'IGHV6-1',
 'IGHV7-4-1',
 'IGHV7-81'}

In [42]:
sorted_df['unique V genes'] = ['%d (%d)' % (a,b) for a, b in zip(sorted_df['unique V genes'],
                                                        sorted_df['unique functional V genes'])]
sorted_df['unique D genes'] = ['%d (%d)' % (a,b) for a, b in zip(sorted_df['unique D genes'],
                                                        sorted_df['unique functional D genes'])]
sorted_df['unique J genes'] = ['%d (%d)' % (a,b) for a, b in zip(sorted_df['unique J genes'],
                                                        sorted_df['unique functional J genes'])]

In [45]:
del sorted_df['unique functional V genes']
del sorted_df['unique functional D genes']
del sorted_df['unique functional J genes']
del sorted_df['prova']

In [46]:
sorted_df

,database,n_seqs,clonotypes,avg seqs/clone,unique V genes,unique D genes,unique J genes,mean (std) of V gene mutation
0,clones_100.100.tab,9233,96,96.18,46 (38),29 (24),6 (6),10.38 (5.20)
1,clones_100.200.tab,17825,96,185.68,47 (40),30 (25),6 (6),9.06 (4.76)
2,clones_100.400.tab,37897,96,394.76,45 (40),30 (25),6 (6),9.88 (4.93)
9,clones_500.100.tab,47764,496,96.30,69 (57),30 (25),6 (6),8.92 (4.57)
10,clones_500.200.tab,102336,496,206.32,70 (58),30 (25),6 (6),9.02 (4.99)
11,clones_500.400.tab,205986,496,415.29,68 (56),30 (25),6 (6),9.88 (4.68)
3,clones_1500.100.tab,162713,1487,109.42,71 (58),30 (25),6 (6),9.18 (4.88)
4,clones_1500.200.tab,301978,1488,202.94,71 (58),30 (25),6 (6),9.52 (4.87)
5,clones_1500.400.tab,589680,1488,396.29,71 (58),30 (25),6 (6),9.31 (4.88)
6,clones_3000.100.tab,291076,2991,97.32,71 (58),30 (25),6 (6),9.54 (4.85)


In [63]:
df[['true_v', 'v_call']].iloc[0:72]
df[df['true_v'] == 'IGHV3-11', ['']]

,sequence_id,sequence_input,functional,in_frame,stop,mutated_invariant,indels,v_call,d_call,j_call,...,v_gene_set,v_gene_set_str,j_gene_set,junc,aa_junc,aa_junction_length,true_clone,true_v,true_d,true_j
74727,clones_500.400.csv_2805183687602977237_6406908...,CAGGTGCAGCTGGTGGAGGCTGGGGGAGGCTTGGTTTAGGTTGGAG...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCGAGAATATTATGGGTACTTGGGGGAGAGTTCTCCTTACCCT...,CARILWVLGGEFSLPWGMDVW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74728,clones_500.400.csv_3789160505661039811_6406908...,CAGCTGCAGCTGGTGGAGTCTGGGGGTGGCTTGGTCAAGCCCGGAG...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCGAAAATATTATGAGTACGCTGGGGGGAGTTCTTCTTACAGC...,CAKIL*VRWGEFFLQRGMDVW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74729,clones_500.400.csv_-1857988558188441498_640690...,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAG...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*02 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCAAAAATAATATGAGTACGTACGGGGGAGTTCTGCCTAGGCT...,CAKII*VRTGEFCLG*GLDAW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74730,clones_500.400.csv_5533581010273220291_6406908...,CCGGTGCAGCTGGTGGAGTCTGGGGGAGCCTTGGTCAAGCCTGGAG...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCAAGAATATTATGAGTACGTATGGGGGAGTTCTGCCTATGCG...,CARIL*VRMGEFCLCDGVDVW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74731,clones_500.400.csv_-7685916871597654520_640690...,CAGATGCAACTGGTGGAGTCTGGGGGAGGGTTGGTCAAGCCTGGAG...,T,None,F,F,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCAAGAATATTATGGGAACGTATGGGGAAGGTCTGCTTATGGA...,CARILWERMGKVCLWIGMDVW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74732,clones_500.400.csv_-6339261542305513109_640690...,CAGGTGCTGCTGGTGGAGTCTGGCGGAGGCTTCGTCAAGCCTGGAA...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*02 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCAAGAGTACTGTGATTACGTTTTGGGGGGTCAGGCGTACACT...,CARVL*LRFGGSGVHFGVDVW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74733,clones_500.400.csv_2288766508160027594_6406908...,CAGGTGTAGCTGGGGGAGTCTGGGGGAGCCTTGGTCAAGCCTGGAG...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ6*01 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCCAGGGTGTTATGTTCACGTATGGGGGAGTTATGCCTATACT...,CARVLCSRMGELCLYCGVDVW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74734,clones_500.400.csv_566800609393056509_64069088...,CAGGTGCAGTTAGTGAAGTCTGAGGGAGGTTTGGTCAAGCCTGGAG...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCGAGAGTAGGGGGATTTCATTTACGGGAGATTTGCTTATACT...,CARVGGFHLREICLYSGADNW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74735,clones_500.400.csv_-3407007909540781604_640690...,CAGGTGCAGCTAGTGGAGTCTGGGGGAGGTTTGGTAAAGCCTGGAG...,F,None,None,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ4*03 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ4},TGTGCGAGAGTAGGGGGATTGCATTCGGGGGAGTTTTGCTTATACT...,CARVGGLHSGEFCLYSGTS,19,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
74736,clones_500.400.csv_107386063713196544_64069088...,CCGGTCCAGCTAGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAG...,F,None,T,None,F,Homsap IGHV3-11*06 F,Homsap IGHD3-16*01 F,Homsap IGHJ6*02 F,...,{IGHV3-11},set(['IGHV3-11']),{IGHJ6},TGTGCGAGAGTATTAGCATTGCGTTTGCGGGCGTTATGCTTATTTT...,CARVLALRLRALCLFYGGDDW,21,6406908891134225977,IGHV3-11,IGHD3-16,IGHJ6
